<img src="./logo_gsi.png" alt="Grupo de Sistemas Inteligentes" width="100px">

# SOIL Tutorial 

This notebook contains a tutorial to learn how to use the SOcial network sImuLator (SOIL) written in Python. 

## Introduction

SOIL is based in 2 main files:
* __soil.py__: It's the main file of SOIL. The network creation, simulation and visualization are done in this file.
+ __settings.json__: This file contains every variable needed in the simulation in order to be modified easily.
- __models__: All the spread models already implemented are stored in this directory as modules.

## Requirements

SOIL requires to install:
* **Python 3** - you can use the Conda distribution
* **NetworkX** - install with conda install networkx or pip install networkx
* **simpy** - install with pip install simpy
* **nxsim** - install with  pip install nxsim
* **Gephi** - Available at https://gephi.org

## Soil.py

### Imports and data initialization

First of all, you need to make all the imports. This simulator is based on [nxsim](https://pypi.python.org/pypi/nxsim), using [networkx](https://networkx.github.io/) for network management. We will also include the models and settings files where the spread models and initialization variables are stored.

In [ ]:
from models import *
from nxsim import NetworkSimulation
# import numpy
from matplotlib import pyplot as plt
import networkx as nx
import settings
import models
import math
import json

### Network creation

Using a parameter provided in the settings file, we can choose what type of network we want to create, as well as the number of nodes and some other parameters. More types of networks can be implemented using [networkx](https://networkx.github.io/).

In [ ]:
if settings.network_params["network_type"] == 0:
    G = nx.complete_graph(settings.network_params["number_of_nodes"])
if settings.network_params["network_type"] == 1:
    G = nx.barabasi_albert_graph(settings.network_params["number_of_nodes"], 10)
if settings.network_params["network_type"] == 2:
    G = nx.margulis_gabber_galil_graph(settings.network_params["number_of_nodes"], None)
# More types of networks can be added here

### Visualization

In order to analyse the results of the simulation. We include them in the topology and a .gexf file is generated. This allows the user to picture the network in [Gephi](https://gephi.org/). A JSON file is also generated to permit further analysis.

The JSON file follows this schema. The file has three depth levels. In the first one we can find the identifier of each agent in the network. Secondly, inside every agent we can observe every attribute that the creator of the model wanted to make visible. In the deepest level the different values of each attribute are
visible.

	{
		"agent_0": {
			"attribute_X": {
				"0": 0,
				"2": 0,
				"4": 1,
				"6": 2,
				...
			}
		},
		"agent_1": {
			"attribute_X": {
				"0": 0,
				"2": 3,
				...
			}
		},
		...		
	}

This is done with the following code:

In [ ]:
def visualization(graph_name):

    for x in range(0, settings.network_params["number_of_nodes"]):
        for attribute in models.networkStatus["agent_%s" % x]:
            emotionStatusAux = []
            for t_step in models.networkStatus["agent_%s" % x][attribute]:
                prec = 2
                output = math.floor(models.networkStatus["agent_%s" % x][attribute][t_step] * (10 ** prec)) / (10 ** prec)  # 2 decimals
                emotionStatusAux.append((output, t_step, t_step + settings.network_params["timeout"]))
            attributes = {}
            attributes[attribute] = emotionStatusAux
            G.add_node(x, attributes)

    print("Done!")

    with open('data.txt', 'w') as outfile:
        json.dump(models.networkStatus, outfile, sort_keys=True, indent=4, separators=(',', ': '))

    nx.write_gexf(G, graph_name+".gexf", version="1.2draft")

That's only the basic visualization. Everything you need can be implemented as well. For example:

In [ ]:
def results(model_name):
    x_values = []
    infected_values = []
    neutral_values = []
    cured_values = []
    vaccinated_values = []

    attribute_plot = 'status'
    for time in range(0, settings.network_params["max_time"]):
        value_infectados = 0
        value_neutral = 0
        value_cured = 0
        value_vaccinated = 0
        real_time = time * settings.network_params["timeout"]
        activity = False
        for x in range(0, settings.network_params["number_of_nodes"]):
            if attribute_plot in models.networkStatus["agent_%s" % x]:
                if real_time in models.networkStatus["agent_%s" % x][attribute_plot]:
                    if models.networkStatus["agent_%s" % x][attribute_plot][real_time] == 1:  ## Infected
                        value_infectados += 1
                        activity = True
                    if models.networkStatus["agent_%s" % x][attribute_plot][real_time] == 0:  ## Neutral
                        value_neutral += 1
                        activity = True
                    if models.networkStatus["agent_%s" % x][attribute_plot][real_time] == 2:  ## Cured
                        value_cured += 1
                        activity = True
                    if models.networkStatus["agent_%s" % x][attribute_plot][real_time] == 3:  ## Vaccinated
                        value_vaccinated += 1
                        activity = True

        if activity:
            x_values.append(real_time)
            infected_values.append(value_infectados)
            neutral_values.append(value_neutral)
            cured_values.append(value_cured)
            vaccinated_values.append(value_vaccinated)
            activity = False

    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)

    infected_line = ax1.plot(x_values, infected_values, label='Infected')
    neutral_line = ax1.plot(x_values, neutral_values, label='Neutral')
    cured_line = ax1.plot(x_values, cured_values, label='Cured')
    vaccinated_line = ax1.plot(x_values, vaccinated_values, label='Vaccinated')
    ax1.legend()
    fig1.savefig(model_name + '.png')
    # plt.show()

### Simulation

The simulation starts with the following code. The user can provide the network topology, the maximum time of simulation, the spread model to be used as well as other parameters.

In [ ]:
agents = settings.environment_params['agent']

print("Using Agent(s): {agents}".format(agents=agents))

if len(agents) > 1:
    for agent in agents:
        sim = NetworkSimulation(topology=G, states=init_states, agent_type=locals()[agent], max_time=settings.network_params["max_time"],
                                num_trials=settings.network_params["num_trials"], logging_interval=1.0, **settings.environment_params)
        sim.run_simulation()
        print(str(agent))
        results(str(agent))
        visualization(str(agent))
else:
    agent = agents[0]
    sim = NetworkSimulation(topology=G, states=init_states, agent_type=locals()[agent], max_time=settings.network_params["max_time"],
                            num_trials=settings.network_params["num_trials"], logging_interval=1.0, **settings.environment_params)
    sim.run_simulation()
    results(str(agent))
    visualization(str(agent))

## Models

### Imports and initialization

In [ ]:
import settings

networkStatus = {}  # Dict that will contain the status of every agent in the network

sentimentCorrelationNodeArray = []
for x in range(0, settings.network_params["number_of_nodes"]):
    sentimentCorrelationNodeArray.append({'id': x})
# Initialize agent states. Let's assume everyone is normal.
init_states = [{'id': 0, } for _ in range(settings.network_params["number_of_nodes"])]
    # add keys as as necessary, but "id" must always refer to that state category

### Base behaviour

Every spread model used in SOIL should extend the base behaviour class.  By doing this the exportation of the attributes values will be automatic. This feature will be explained in the Spread Models section. The class looks like this:

In [ ]:
import settings
from nxsim import BaseNetworkAgent


class BaseBehaviour(BaseNetworkAgent):

    def __init__(self, environment=None, agent_id=0, state=()):
        super().__init__(environment=environment, agent_id=agent_id, state=state)
        self._attrs = {}

    @property
    def attrs(self):
        now = self.env.now
        if now not in self._attrs:
            self._attrs[now] = {}
        return self._attrs[now]

    @attrs.setter
    def attrs(self, value):
        self._attrs[self.env.now] = value

    def run(self):
        while True:
            self.step(self.env.now)
            yield self.env.timeout(settings.network_params["timeout"])

    def step(self, now):
        networkStatus['agent_%s'% self.id] = self.to_json()

    def to_json(self):
        final = {}
        for stamp, attrs in self._attrs.items():
            for a in attrs:
                if a not in final:
                    final[a] = {}
                final[a][stamp] = attrs[a]
        return final

### Spread models

Every model to be implemented must include an init and a step function.  Depending on your model, you would need different attributes. If you want them to be automatic exported for a further analysis, you must name them like this *self.attrs['name_of_attribute']*. Moreover, the last thing you should do inside the step function is call the following method *super().step(now)*. This call will store the values.

Some other tips:
* __self.state['id']__: To check the id of the current agent/node.
* __self.get_neighboring_agents(state_id=x)__: Returns the neighbours agents/nodes with the id provided

An example of a spread model already implemented and working:



In [ ]:
import settings
import random
import numpy as np


class ControlModelM2(BaseBehaviour):

    # Init infected
    init_states[random.randint(0, settings.network_params["number_of_nodes"]-1)] = {'id': 1}
    init_states[random.randint(0, settings.network_params["number_of_nodes"]-1)] = {'id': 1}

    # Init beacons
    init_states[random.randint(0, settings.network_params["number_of_nodes"]-1)] = {'id': 4}
    init_states[random.randint(0, settings.network_params["number_of_nodes"]-1)] = {'id': 4}

    def __init__(self, environment=None, agent_id=0, state=()):
        super().__init__(environment=environment, agent_id=agent_id, state=state)

        self.prob_neutral_making_denier = np.random.normal(environment.environment_params['prob_neutral_making_denier'],
                                                           environment.environment_params['standard_variance'])

        self.prob_infect = np.random.normal(environment.environment_params['prob_infect'],
                                            environment.environment_params['standard_variance'])

        self.prob_cured_healing_infected = np.random.normal(environment.environment_params['prob_cured_healing_infected'],
                                                            environment.environment_params['standard_variance'])
        self.prob_cured_vaccinate_neutral = np.random.normal(environment.environment_params['prob_cured_vaccinate_neutral'],
                                                             environment.environment_params['standard_variance'])

        self.prob_vaccinated_healing_infected = np.random.normal(environment.environment_params['prob_vaccinated_healing_infected'],
                                                                 environment.environment_params['standard_variance'])
        self.prob_vaccinated_vaccinate_neutral = np.random.normal(environment.environment_params['prob_vaccinated_vaccinate_neutral'],
                                                                  environment.environment_params['standard_variance'])
        self.prob_generate_anti_rumor = np.random.normal(environment.environment_params['prob_generate_anti_rumor'],
                                                         environment.environment_params['standard_variance'])

    def step(self, now):

        if self.state['id'] == 0:  # Neutral
            self.neutral_behaviour()
        elif self.state['id'] == 1:  # Infected
            self.infected_behaviour()
        elif self.state['id'] == 2:  # Cured
            self.cured_behaviour()
        elif self.state['id'] == 3:  # Vaccinated
            self.vaccinated_behaviour()
        elif self.state['id'] == 4:  # Beacon-off
            self.beacon_off_behaviour()
        elif self.state['id'] == 5:  # Beacon-on
            self.beacon_on_behaviour()

        self.attrs['status'] = self.state['id']
        super().step(now)

    def neutral_behaviour(self):

        # Infected
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        if len(infected_neighbors) > 0:
            if random.random() < self.prob_neutral_making_denier:
                self.state['id'] = 3   # Vaccinated making denier

    def infected_behaviour(self):

        # Neutral
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_infect:
                neighbor.state['id'] = 1  # Infected

    def cured_behaviour(self):

        # Vaccinate
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_cured_vaccinate_neutral:
                neighbor.state['id'] = 3  # Vaccinated

        # Cure
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors:
            if random.random() < self.prob_cured_healing_infected:
                neighbor.state['id'] = 2  # Cured

    def vaccinated_behaviour(self):

        # Cure
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors:
            if random.random() < self.prob_cured_healing_infected:
                neighbor.state['id'] = 2  # Cured

        # Vaccinate
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_cured_vaccinate_neutral:
                neighbor.state['id'] = 3  # Vaccinated

        # Generate anti-rumor
        infected_neighbors_2 = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors_2:
            if random.random() < self.prob_generate_anti_rumor:
                neighbor.state['id'] = 2  # Cured

    def beacon_off_behaviour(self):
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        if len(infected_neighbors) > 0:
            self.state['id'] == 5  # Beacon on

    def beacon_on_behaviour(self):

        # Cure (M2 feature added)
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors:
            if random.random() < self.prob_generate_anti_rumor:
                neighbor.state['id'] = 2  # Cured
            neutral_neighbors_infected = neighbor.get_neighboring_agents(state_id=0)
            for neighbor in neutral_neighbors_infected:
                if random.random() < self.prob_generate_anti_rumor:
                    neighbor.state['id'] = 3  # Vaccinated
            infected_neighbors_infected = neighbor.get_neighboring_agents(state_id=1)
            for neighbor in infected_neighbors_infected:
                if random.random() < self.prob_generate_anti_rumor:
                    neighbor.state['id'] = 2  # Cured

        # Vaccinate
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_cured_vaccinate_neutral:
                neighbor.state['id'] = 3  # Vaccinated

## Settings.json

This file contains all the variables that can be modified from the simulation. In case of implementing a new spread model, the new variables should be also included in this file.

In [ ]:
[
  {
    "network_type": 1,
    "number_of_nodes": 1000,
    "max_time": 50,
    "num_trials": 1,
    "timeout": 2
  },

  {
    "agent": ["BaseBehaviour","SISaModel","ControlModelM2"],


    "bite_prob": 0.01,
    "heal_prob": 0.01,

    "innovation_prob": 0.001,
    "imitation_prob": 0.005,

    "outside_effects_prob": 0.2,
    "anger_prob": 0.06,
    "joy_prob": 0.05,
    "sadness_prob": 0.02,
    "disgust_prob": 0.02,

    "enterprises": ["BBVA", "Santander", "Bankia"],

    "tweet_probability_users": 0.44,
    "tweet_relevant_probability": 0.25,
    "tweet_probability_about": [0.15, 0.15, 0.15],
    "sentiment_about": [0, 0, 0],

    "tweet_probability_enterprises": [0.3, 0.3, 0.3],

    "neutral_discontent_spon_prob": 0.04,
    "neutral_discontent_infected_prob": 0.04,
    "neutral_content_spon_prob": 0.18,
    "neutral_content_infected_prob": 0.02,

    "discontent_neutral": 0.13,
    "discontent_content": 0.07,
    "variance_d_c": 0.02,

    "content_discontent": 0.009,
    "variance_c_d": 0.003,
    "content_neutral": 0.088,

    "standard_variance": 0.055,


    "prob_neutral_making_denier": 0.035,

    "prob_infect": 0.075,

    "prob_cured_healing_infected": 0.035,
    "prob_cured_vaccinate_neutral": 0.035,

    "prob_vaccinated_healing_infected": 0.035,
    "prob_vaccinated_vaccinate_neutral": 0.035,
    "prob_generate_anti_rumor": 0.035
  }
]

## Model Library

To test this simulator in all the experiments we have used the Albert
Barabasi Graph [34] to automatically generate the network and the con-
nections among the agents due it is one of the most suitable graphs to
recreate social networks.

Using different human behaviour models we will recreate the different
decisions of each agent.

Moreover there are some parameters regarding the basic simulation that
have to be settled. In addition, more parameters will be needed depend-
ing on the spread model used for the experiment.

### Spread Model M2

This model is based on the New Spread Model
M2 [1] which also refers to the cascade model [2]. Agents, usually Twit-
ter users, have four states regarding a rumour: neutral (initial state),
infected, vaccinated and cured.

An agent becomes: infected when believes the rumour; vaccinated when is
influenced before being infected by a cured or already vaccinated agent
and cured when after becoming infected the agent is influenced by a
vaccinated/cured user.

After a certain period of time, a random infected user develops an anti-
rumour and spreads it to its neighbours in order to vaccinate the neutral
and cure the infected ones.

This model includes the fact that infected users who made a mistake
believing in the rumour will not be in favour of spreading theirs mistakes
through the network. Therefore, only vaccinated users will spread anti-
rumours. The probability of making a denier and becoming vaccinated
when a neutral user has an infected neighbour and the first already had
information about the rumour being false.

* [1] E. Serrano and C. A. Iglesias. “Validating viral marketing
strategies in Twitter via agent-based social simulation”. In:
Expert Systems with Applications 50.1 (2016),
* [2] L. Weng et al. “Virality prediction and community structure
in social networks”. In: Scientific Reports 3 (2013).

### Control model M2,2

This model is based on the New Control Model
M2,2 [1]. It includes the use of beacons, special agents, that represent
an authority which can work against the rumour once it is detected. It
only has two states: on or off. Beacons will switch to on status when they
detect the misinformation in an infected neighbour agent.
Once the beacon is activated, they will try to cure and vaccinate other
agents starting a anti-rumour. Therefore this model also takes into ac-
count that infected users might not admit a previous mistake.

* [1] E. Serrano and C. A. Iglesias. “Validating viral marketing
strategies in Twitter via agent-based social simulation”. In:
Expert Systems with Applications 50.1 (2016),

### SISa Model

The SISa model of infection is already included in the simulator. Its the evolution of the classic disease spread Susceptible-Infective-Susceptible (SIS) model [1, 2].

The SISa model is proposed by [3] and the main new feature is considering the spontaneous generation process of sentiment. This model has two assumptions: first, a susceptible agent who is close and more exposed to the infected has a higher probability of infection that other agent; second, the number of infected agents does not affect the probability of recovery.

Based on some recent implementations of the SISa model [3], every agent can be in three states: neutral (initial), content and discontent.

All the transitions between every different state are allowed depending on customizable probabilities. This model includes the fact that an agent will be more likely to change state as the number of neighbours with this state increases.

* [1] P. Weng and X.-Q. Zhao. “Spreading speed and traveling waves for a multi-type SIS epidemic model”. In: Journal of Differential Equations 229.1 (2006)

* [2] P. V. Mieghem. “Epidemic phase transition of the SIS type in networks”. In: A Letters Journal Exploring the Frontiers of Physics 97.4 (2012).
* [3] A. L. Hill et al. “Emotions as infectious diseases in a large social network: the SISa model”. In: Proceedings of the Royal Society of London B: Biological Sciences 277.1701 (2010),

### Big Market Model

As stated in several papers [2–4], social networks like Twitter are the perfect scenario to study the propagation of ideas, sentiments and marketing strategies. In this scenario several enterprises want to take advantage of social networks to promote their companies and connect with their clients.

The goal of this model [1] is to recreate the behaviour of several enterprises in a social network. Following the example of HashtKat, we want to measure the effect of different marketing strategies in social networks.
Depending on the sentiment towards an enterprise the user will post positive or negative tweets about these enterprises. The fact that an user can increase its probabilities of posting a relevant tweet about a certain
company depending on its sentiment towards it is also considered.
In this model the number of enterprises as well as tweet rate probabilities of both companies and users can be changed.

* [1] E. Serrano and C. A. Iglesias. “Validating viral marketing
strategies in Twitter via agent-based social simulation”. In:
Expert Systems with Applications 50.1 (2016)
* [2] B. A. Huberman et al. “Social Networks that Matter: Twitter
Under the Microscope”. In: Social Science Research Network
(2008).
* [3] M. Cha et al. “Measuring User Influence in Twitter: The
Million Follower Fallacy.” In: ICWSM 10.10-17 (2010),
* [4] M. Bulearca and S. Bulearca. “Twitter: a viable marketing
tool for SMEs?” In: Global business and management research
2.4 (2010),

### Sentiment Correlation Model

With this model we want to study
the influence of different sentiments in a social network. In order to do so, we base our model on the research made by [1]. In this paper the authors found out that in a social network (in this case Weibo) the correlation
of anger is significantly higher than joy and sadness meaning that the anger sentiment would occasionally spread faster than the others.

They also confirmed some intuitive ideas such as a pair of users who have higher interactions are more likely to be influenced by each other, and that users with more friends would influence their neighbours more than other agents.

In this simulation we have four emotions: anger, joy, sadness and disgust.

Using the probabilities extracted from the dataset used by [1] we can visualise the graph and confirm the conclusions of the paper. Anger sentiment propagation rate is much higher than any other. Joy sentiment also spreads easily to the neighbours. However, sadness and disgust propagation rate is really small, few neighbours get affected by them.

* [1] R. Fan et al. “Anger is More Influential Than Joy: Sentiment
Correlation in Weibo”. In: CoRR abs/1309.2402 (2013).

### Bass Model

Even though Bass Model can be applied to many appli-
cations [57–60] it can be used to study the diffusion of information as
well.
This model is based on the implementation proposed by Rand and Wilen-
sky [13]. In this scenario there are only two states: unaware (initial) and
aware. For this simulation we assume that agents can only change status
from advertising (outside effects) and word of mouth (information inside
the network).
The probability of being affected by imitation (word of mouth effect)
increases as a function of the agent aware neighbours. In this model once
the user changes to aware status it remains in this state for the whole
simulation.

* F. M. Bass. “A New Product Growth for Model ConsumerDurables”. In: Management Science 15.5 (1969),
W. Dodds. “An Application of the Bass Model in Long-TermNew Product Forecasting”. In: Journal of Marketing Research
10.3 (1973),
*  F. Douglas Tigert. “The Bass New Product Growth Model: A Sensitivity Analysis for a High Technology Product”. In: Journal of Marketing 45.4 (1981),
* Z. Jiang et al. “Virtual Bass Model and the left-hand data-truncation bias in diffusion of innovation studies”. In: International Journal of Research in Marketing 23.1 (2006), 

### Independent Cascade Model

As stated by Rand and Wilensky [1], the Independent Cascade Model [61] suits better the case we want to
study as it is more appropriate for social networks than the Bass Model.

In this scenario we also have two states: unaware (initial) and aware. The new feature in this model is that one agent will only get infected once at least one neighbour became aware the previous time step. There is also
a probability of becoming aware by outside effects (innovation).

This new feature can be explained intuitively, one agent will have more influence on another if the first just infected and wants to spread the new information he acquired.


* [1] W. Rand and U. Wilensky. An Introduction to Agent-Based Modeling: Modeling Natural, Social, and Engineered Complex Systems with NetLogo. MIT Press, 2015.
* [2] J. Goldenberg et al. “Talk of the Network: A Complex Systems Look at the Underlying Process of Word-of-Mouth”. In: Marketing Letters 12.3 (2001),


## Copyright

SOIL has been developed by the Intelligent Systems Group, Universidad Politécnica de Madrid, 2016-2017.

@Copyright Universidad Politécnica de Madrid, 2016-2017
 <img src="./logo_gsi.png" alt="Grupo de Sistemas Inteligentes" width="100px">